# Simple LLM Application

*Taken from the [LangChain documentation](https://python.langchain.com/v0.2/docs/tutorials/llm_chain/).*

In this quickstart we'll show you how to build a simple LLM application. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

## Concepts

- Using language models
- Using PromptTemplates and OutputParsers
- Chaining a PromptTemplate + LLM + OutputParser using LangChain
- Debugging and tracing your application using LangSmith
- Deploying your application with LangServe

## Setup

### Jupyter Notebook

This guide (and most of the other guides in the documentation) uses Jupyter notebooks and assumes the reader is as well. Jupyter notebooks are perfect for learning how to work with LLM systems because oftentimes things can go wrong (unexpected output, API down, etc) and going through guides in an interactive environment is a great way to better understand them.

This and other tutorials are perhaps most conveniently run in a Jupyter notebook.

### Installation

We're deviating from the tutorial a bit here and running this on a local llm. We host an API with ollama and below is the method we'll be using to access it. We also need `langchain` installed. We will not be using `langsmith` as it's a paid service.

`conda install langchain -c conda-forge`

## Detailed Walkthrough

In this guide we will build an application to translate user input from one language to another.

### Using Language Models 

First up, let's learn how to use a language model by itself. LangChain supports many different language models that you can use interchangably - we're using llama3 with the `ollama` API.

In [2]:
from langchain_community.chat_models import ChatOllama

model = ChatOllama(model="llama3:instruct", base_url="http://localhost:11434")

Let's first use the model directly. `ChatModel`s are instances of LangChain "Runnables", which means they expose a standard interface for interacting with them. To just simply call the model, we can pass in a list of messages to the `.invoke` method.

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

model.invoke(messages)

AIMessage(content='Ciao!', response_metadata={'model': 'llama3:instruct', 'created_at': '2024-05-27T17:07:49.4520412Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 5078619500, 'load_duration': 4859503700, 'prompt_eval_count': 23, 'prompt_eval_duration': 149886000, 'eval_count': 4, 'eval_duration': 66769000}, id='run-11cf6abc-1255-4639-bae1-338f9a762d98-0')

### Output Parsers

Notice that the response from the model is an `AIMessage`. This contains a string response along with other metadata about the response. Oftentimes we may just want to work with the string response. We can parse out just this response by using a simple output parser.

We first import the simple output parser.

In [6]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

One way to use it is to use it by itself. For example, we could save the result of the language model call and then pass it to the parser.

In [7]:
result = model.invoke(messages)
parser.invoke(result)

'Ciao!'

More commonly, we can "chain" the model with this output parser. This means this output parser will get called everytime in this chain. This chain takes on the input type of the language model (string or list of message) and returns the output type of the output parser (string).

We can easily create the chain using the `|` operator. The `|` operator is used in LangChain to combine two elements together.

In [8]:
chain = model | parser
chain.invoke(messages)

'Ciao!'

The chain has two steps: first the language model is called, then the result of that is passed to the output parser.

### Prompt Templates

Right now we are passing a list of messages directly into the language model. Where does this list of messages come from? Usually it constructed from a combination of user input and application logic. This application logic usually takes the raw user input and transforms it into a list of messages ready to pass to the language model. Common transformations include adding a system message or formatting a template with the user input.

PromptTemplates are a concept in LangChain designed to assist with this transformation. They take in raw user input and return data (a prompt) that is ready to pass into a language model. 

Let's create a PromptTemplate here. It will take in two user variables:

- `language`: The language to translate text into
- `text`: The text to translate

In [9]:
from langchain_core.prompts import ChatPromptTemplate

First, let's create a string that we will format to be the system message:

In [10]:
system_template = "Translate the following into {language}:"

Next, we can create the PromptTemplate. This will be a combination of the `system_template` as well as a simpler template for where the put the text

In [11]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

The input to this prompt template is a dictionary. We can play around with this prompt template by itself to see what it does by itself

In [12]:
result = prompt_template.invoke({"language": "italian", "text": "hi"})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into italian:'), HumanMessage(content='hi')])

We can see that it returns a `ChatPromptValue` that consists of two messages. If we want to access the messages directly we do:

In [13]:
result.to_messages()

[SystemMessage(content='Translate the following into italian:'),
 HumanMessage(content='hi')]

We can now combine this with the model and the output parser from above. This will chain all three components together.

In [15]:
chain = prompt_template | model | parser
chain.invoke({"language": "italian", "text": "hi"})

'Ciao!'

## Serving with LangServe

Now that we've built an application, we need to serve it. That's where LangServe comes in. LangServe helps developers deploy LangChain chains as a REST API. You do not need to use LangServe to use LangChain, but in this guide we'll show how you can deploy your app with LangServe.

While the first part of this guide was intended to be run in a Jupyter Notebook or script, we will now move out of that. We will be creating a Python file and then interacting with it from the command line.

Install with:

```ps
pip install "langserve[all]"
```

### Server

To create a server for our application we'll make a `serve.py` file. This will contain our logic for serving our application. It consists of three things:

1. The definition of our chain that we just built above
2. Our FastAPI app
3. A definition of a route from which to serve the chain, which is done with `langserve.add_routes`

```python
#!/usr/bin/env python
from typing import List

from fastapi import FastAPI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langserve import add_routes

# 1. Create prompt template
system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages([
    ('system', system_template),
    ('user', '{text}')
])

# 2. Create model
model = ChatOllama(model="llama3:instruct", base_url="http://localhost:11434")

# 3. Create parser
parser = StrOutputParser()

# 4. Create chain
chain = prompt_template | model | parser


# 4. App definition
app = FastAPI(
  title="LangChain Server",
  version="1.0",
  description="A simple API server using LangChain's Runnable interfaces",
)

# 5. Adding chain route

add_routes(
    app,
    chain,
    path="/chain",
)

if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="localhost", port=8000)
```

And that's it! If we execute this file:

```ps
python serve.py
```

we should see our chain being served at http://localhost:8000.

### Playground

Every LangServe service comes with a simple built-in UI for configuring and invoking the application with streaming output and visibility into intermediate steps. Head to http://localhost:8000/chain/playground/ to try it out! Pass in the same inputs as before - `{"language": "italian", "text": "hi"}` - and it should respond same as before.

### Client

Now let's set up a client for programmatically interacting with our service. We can easily do this with the `langserve.RemoteRunnable`. Using this, we can interact with the served chain as if it were running client-side.

*Note: we're already running client-side so this is a bit of a redundant step. Proof of concept, I guess:*

In [18]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/chain/")
remote_chain.invoke({"language": "italian", "text": "hi"})

'Ciao'

## Conclusion

That's it! In this tutorial we've walked through creating our first simple LLM application. We've learned how to work with language models, how to parse their outputs, how to create a prompt template, how to get great observability into chains you create with LangSmith, and how to deploy them with LangServe.

This just scratches the surface of what you will want to learn to become a proficient AI Engineer. Luckily - we've got a lot of other resources!